# Download data

In [12]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime

start = datetime.datetime(2011,1,1)
end = datetime.date.today()

tcs = data.DataReader("RELIANCE.NS",'yahoo',start,end)

tcs.to_csv("RELIANCE.csv",index=True,header=True)

#print(tcs.head())

#print(tcs.tail())

                  High         Low        Open       Close      Volume  \
Date                                                                     
2011-01-03  528.242798  521.358032  527.499817  522.843933   4804792.0   
2011-01-04  534.879883  523.685974  525.023315  533.493042  10163093.0   
2011-01-05  539.882446  529.976379  534.929382  532.849121  11972802.0   
2011-01-06  540.575867  532.229980  533.938782  537.703125  10043704.0   
2011-01-07  538.768005  524.032715  535.573303  527.697937   8249359.0   

             Adj Close  
Date                    
2011-01-03  480.088593  
2011-01-04  489.866913  
2011-01-05  489.275665  
2011-01-06  493.732635  
2011-01-07  484.545654  
                   High          Low    Open        Close      Volume  \
Date                                                                    
2021-03-04  2189.949951  2157.699951  2180.0  2175.850098   9892597.0   
2021-03-05  2211.949951  2153.050049  2156.0  2178.699951  11773630.0   
2021-03-08  22

# ti_calculator

In [13]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


c_name = "RELIANCE"

def get_rsi(sdata,m,mem):
    neg = 0
    pos = 0
    RS = 0
    RSI = 100

    upcloses = 0
    downcloses = 0

    n = m
    k = m-1

    for p in range(mem):
       diff = sdata[n,3] - sdata[k,3]
       if (diff>=0):
        upcloses = upcloses + diff
        pos = pos+1
       else:
        downcloses = downcloses + diff
        neg = neg+1
    
       n = n-1
       k = k-1
 
    downcloses = -downcloses
    if(neg == 0):
        return 100
    else:
        RS = (upcloses*neg)/(downcloses*pos)

    
    RSI = 100 - (100/(1+RS))

    return RSI
    

def get_mfi(sdata,m,mem):
    neg = 0
    pos = 0
    MFR = 0
    MFI = 100

    pmflow = 0
    nmflow = 0

    n = m
    k = m-1

    for p in range(mem):

       typ_pricec = (sdata[n,0] + sdata[n,1] + sdata[n,3])/3
       typ_pricep = (sdata[k,0] + sdata[k,1] + sdata[k,3])/3
     
      # print(typ_price,sdata[n,0],sdata[n,1],sdata[n,3])

       if (typ_pricec>=typ_pricep):
         pmflow = pmflow + ((sdata[n,4])*(typ_pricec))
         pos = pos+1
       else:
         nmflow = nmflow + ((sdata[n,4])*(typ_pricec))
         neg = neg+1
       
       n = n-1
       k = k-1
    
    if(neg == 0):
        return 100
    else:
        MFR = pmflow/nmflow

    MFI = 100 - (100/(1+MFR))

    return MFI

def get_ema(sdata,m,mem,EMAp):
   
   EMA = sdata[m,3]*(2/(1+mem)) + (1-(2/(1+mem)))*EMAp 
   print("EMA",EMA)

   return EMA

def get_so(sdata,m,mem):
   
   SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100
   print("SO",SO)
   return SO



with open('./'+ c_name +'.csv', 'r') as csvFile:
    data = list(csv.reader(csvFile,delimiter=','))

memory = 14

#print(data[0])

sdat = pd.DataFrame(data[1:])
sdat2 = sdat.drop([0,6],axis = 1)
sdat3 = np.array(sdat2,dtype=np.float32)


#print(sdat3[0])

df1 = pd.DataFrame(columns=['Open','High','Low','Close','Volume','RSI','MFI','EMA','SO','CloseNext'])

df2 = pd.DataFrame(columns=['Close','RSI','MFI','EMA','SO','CloseNext'])

arr = np.array(df1.values)


print("Printing j\n")

EMAp = 0
acc = 0

for i in range(memory):
    acc = acc + sdat3[i,3]


EMAp = acc / memory

#for i in range(len(sdat3) -memory):
for i in range(len(sdat3)-1 -memory):
    j = i + memory

    RSI = get_rsi(sdat3,j,memory)
    #print("RSI:",RSI)

    MFI = get_mfi(sdat3,j,memory)
    #print("MFI:",MFI)

    EMA = get_ema(sdat3,j,memory,EMAp)
    EMAp = EMA

    SO = get_so(sdat3,j,memory)

    N_close = sdat3[j+1,3]

    rec1 = [sdat3[j,2],sdat3[j,0],sdat3[j,1],sdat3[j,3],sdat3[j,4],RSI,MFI,EMA,SO,N_close]
    rec2 = [sdat3[j,3],RSI,MFI,EMA,SO,N_close]

    if(sdat3[j,4]!=0):
        d1 = {"Open":sdat3[j,2],"High":sdat3[j,0],"Low":sdat3[j,1],"Close":sdat3[j,3],"Volume":sdat3[j,4],"RSI":RSI,"MFI":MFI,"EMA":EMA,"SO":SO,"CloseNext":N_close}
        df1.loc[i] = rec1
        df2.loc[i] = rec2



df1.to_csv(c_name + "_l.csv",index=True,header=True)
df2.to_csv(c_name +"_s.csv",index=True,header=True)


    #rec4 = pd.DataFrame({'Close':sdat3[j,3],'RSI':RSI,'MFI':MFI,'EMA':EMA,'SO':SO,'CloseNext':N_close})

['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close']
[5.2824280e+02 5.2135803e+02 5.2749982e+02 5.2284393e+02 4.8047920e+06]
Printing j

RSI: 54.90166114368036
MFI: 37.582003229273056
EMA 506.60904119582403
SO 70.76919078826904
RSI: 51.8231673291723
MFI: 30.253539302394074
EMA 503.22628134106833
SO 23.584870994091034
RSI: 49.922294984111375
MFI: 21.880954434031565
EMA 499.42942348387385
SO 7.839744538068771
RSI: 53.55735560162431
MFI: 13.98096027065263
EMA 495.0953725857636
SO 9.85415056347847
RSI: 52.25409350956796
MFI: 13.454348291720194
EMA 489.3777856355264
SO 25.056982040405273
RSI: 48.89427611129981
MFI: 13.288547579653397
EMA 484.8385999270916
SO 62.91266083717346
RSI: 48.23983026676102
MFI: 12.786901846263362
EMA 479.3328697740523
SO 17.752160131931305
RSI: 53.14716063481058
MFI: 13.411591941409725
EMA 476.2650862586058
SO 67.96115636825562
RSI: 55.59004830525287
MFI: 18.616310338679213
EMA 475.09886170407293
SO 88.923180103302
RSI: 53.90866080467446
MFI: 18.62187627

RSI: 38.405431809814786
MFI: 34.31086397855461
EMA 438.3543148209784
SO 48.621487617492676
RSI: 40.00285838320405
MFI: 34.9883954847425
EMA 437.43509611958757
SO 53.845930099487305
RSI: 44.51897884388393
MFI: 35.608462201712356
EMA 437.5729166369759
SO 82.70434141159058
RSI: 44.37970442170765
MFI: 42.27856600290859
EMA 438.5674017754833
SO 76.66685581207275
RSI: 40.876568104926285
MFI: 40.125966697352624
EMA 437.01550308823136
SO 10.437457263469696
RSI: 38.87017112939139
MFI: 40.6058122215588
EMA 436.0964845124047
SO 23.610909283161163
RSI: 36.656880544971244
MFI: 38.98186058186492
EMA 433.8173886281986
SO 7.707556337118149
RSI: 34.51737195226232
MFI: 38.89062338179243
EMA 432.30115657022003
SO 68.53954195976257
RSI: 36.16881271009857
MFI: 45.793431341836666
EMA 432.16260961997193
SO 78.62581014633179
RSI: 41.489890011198725
MFI: 47.155682940946
EMA 430.99579080475695
SO 12.954296171665192
RSI: 44.251130267862784
MFI: 42.47586519449565
EMA 429.882187805529
SO 48.64877760410309
RSI: 43.

RSI: 53.23837400327948
MFI: 34.18972618232627
EMA 395.7100973728348
SO 83.27821493148804
RSI: 53.35823754771981
MFI: 33.40926807775179
EMA 395.7152427556756
SO 24.437056481838226
RSI: 50.66685203738259
MFI: 33.933672539361694
EMA 396.58153476976264
SO 89.63092565536499
RSI: 52.37451037446188
MFI: 38.7440912076552
EMA 396.9823049059297
SO 60.64985990524292
RSI: 48.240800176700226
MFI: 44.340642143773444
EMA 397.53106317758704
SO 40.4908686876297
RSI: 49.6962087300457
MFI: 45.31729194814451
EMA 395.96269616015877
SO 18.96073669195175
RSI: 45.624164351824604
MFI: 45.4563521605167
EMA 393.07460359922095
SO 17.44203120470047
RSI: 44.51952048030297
MFI: 44.92631625126232
EMA 388.7092405997936
SO 9.210436791181564
RSI: 46.12394633516966
MFI: 37.44479909920667
EMA 385.90333078544614
SO 85.20014882087708
RSI: 50.02637338081408
MFI: 45.7577390722699
EMA 383.44512626405333
SO 41.463643312454224
RSI: 44.98473227471372
MFI: 45.73968145415592
EMA 381.8595147022837
SO 85.96775531768799
RSI: 43.294865

RSI: 41.27041626646339
MFI: 36.57500681839686
EMA 356.5657952524004
SO 60.54455041885376
RSI: 36.147308311615014
MFI: 26.62896022169079
EMA 354.017195078122
SO 11.340232193470001
RSI: 29.755874780488654
MFI: 26.845573954395277
EMA 351.8315243085911
SO 54.31672930717468
RSI: 33.56812440020593
MFI: 20.183791834263346
EMA 349.5707479880185
SO 31.2820702791214
RSI: 34.003929370495015
MFI: 19.84865969477559
EMA 348.2090769672202
SO 60.995638370513916
RSI: 32.02518737465839
MFI: 25.07892293201246
EMA 347.25350228435127
SO 63.317930698394775
RSI: 36.17772888081278
MFI: 24.053505187430133
EMA 346.88431786518777
SO 64.62572813034058
RSI: 35.291114457974956
MFI: 23.84088798623533
EMA 346.2737818633711
SO 14.826548099517822
RSI: 35.428179109723445
MFI: 23.57314253656783
EMA 345.46397439226536
SO 70.19206881523132
RSI: 40.038436482213264
MFI: 29.644326971437152
EMA 345.32018568423416
SO 66.18690490722656
RSI: 42.7716351500496
MFI: 30.756963763891846
EMA 344.92480464378417
SO 39.25953507423401
RSI:

RSI: 31.631157311890306
MFI: 56.54550079918525
EMA 410.5128092375841
SO 83.85818004608154
RSI: 30.581502365492895
MFI: 51.23858708171915
EMA 409.8584686895
SO 27.536121010780334
RSI: 30.394470032939154
MFI: 57.51356616066024
EMA 409.5654458134521
SO 37.15639114379883
RSI: 31.9059301448453
MFI: 55.41341037048877
EMA 408.61146010212724
SO 8.180327713489532
RSI: 33.56845416447652
MFI: 46.90184408117664
EMA 407.302576131479
SO 22.713102400302887
RSI: 34.91506102593995
MFI: 59.876323016404804
EMA 406.5743582983235
SO 75.26307702064514
RSI: 34.76527560013382
MFI: 53.78226044337604
EMA 405.3818917752137
SO 30.94172477722168
RSI: 35.818319270081844
MFI: 58.83386254415488
EMA 405.19374142653936
SO 94.65340375900269
RSI: 34.28943092565757
MFI: 56.23693142436735
EMA 404.6575495358133
SO 51.500290632247925
RSI: 30.450749794809695
MFI: 57.20713643769428
EMA 404.222565880257
SO 27.825936675071716
RSI: 27.72465604389066
MFI: 58.438170649094985
EMA 403.8984120766915
SO 73.44381213188171
RSI: 39.109903

...

RSI: 53.84168133147568
MFI: 40.22287051722312
EMA 1272.8740902080845
SO 65.50183892250061
RSI: 48.76983428619712
MFI: 46.01844431607811
EMA 1272.7499764876316
SO 36.363229155540466
RSI: 52.09145540186779
MFI: 46.99130800440204
EMA 1269.9149373049058
SO 10.983798652887344
RSI: 49.769710790145176
MFI: 47.7192653651893
EMA 1265.9653740496685
SO 19.346708059310913
RSI: 50.62258120672955
MFI: 48.47894581735852
EMA 1266.2473020409627
SO 91.05239510536194
RSI: 52.76231402004497
MFI: 48.777839153434584
EMA 1266.3265354667512
SO 49.15232956409454
RSI: 50.54478031298343
MFI: 57.28053201456438
EMA 1266.751828784726
SO 21.428488194942474
RSI: 55.25856593873581
MFI: 64.17206795965947
EMA 1266.9817281759292
SO 11.161761730909348
RSI: 50.244496199236536
MFI: 57.608196693002725
EMA 1266.5998219712221
SO 30.026254057884216
RSI: 49.63666435136877
MFI: 57.24194093476296
EMA 1268.5010143281843
SO 89.35617804527283
RSI: 52.57696755345209
MFI: 62.45509505732282
EMA 1268.6760092927598
SO 10.882455110549927
R

RSI: 71.35564614134366
MFI: 76.92657850277288
EMA 1490.6377886190933
SO 20.884451270103455
RSI: 70.12850049825538
MFI: 81.60442980208956
EMA 1499.2339675844644
SO 70.87631225585938
RSI: 68.65993786122014
MFI: 81.65805709461883
EMA 1508.0642302398692
SO 80.94006180763245
RSI: 63.609305624936994
MFI: 79.4800060684806
EMA 1511.8669617808032
SO 9.67751294374466
RSI: 66.49373813164166
MFI: 84.4397070506382
EMA 1519.8317379704463
SO 25.36713182926178
RSI: 65.56095629716623
MFI: 79.31138802015083
EMA 1525.7307191316786
SO 29.439201951026917
RSI: 60.550786782712166
MFI: 72.7702027897093
EMA 1527.382638872455
SO 43.31416189670563
RSI: 65.64946109655136
MFI: 73.2311445944398
EMA 1528.5699504342526
SO 19.160743057727814
RSI: 62.86045693989104
MFI: 73.27865104558225
EMA 1530.133898449269
SO 51.301443576812744
RSI: 57.78823761658666
MFI: 71.75204457494141
EMA 1533.8271455831166
SO 84.46577787399292
RSI: 53.805231036802994
MFI: 63.969519856975516
EMA 1535.6213028647428
SO 40.10314345359802
RSI: 49.8

RSI: 72.03387864739892
MFI: 65.21294069130931
EMA 1120.6010235708231
SO 7.365556061267853
RSI: 77.93938682852328
MFI: 64.5262947638885
EMA 1125.4651579280467
SO 54.29931879043579
RSI: 77.81047647368288
MFI: 64.39797592890389
EMA 1137.0706824438903
SO 83.93555879592896
RSI: 72.10263570200101
MFI: 62.49504258022809
EMA 1149.7440217899132
SO 75.9999930858612
RSI: 73.24207041938534
MFI: 52.49606636680093
EMA 1159.8756619835497
SO 83.3522617816925
RSI: 73.95883819459829
MFI: 66.87292844202256
EMA 1185.3316772347014
SO 74.91167783737183
RSI: 73.43041420763485
MFI: 72.79269741347255
EMA 1208.3577498273662
SO 54.070478677749634
RSI: 71.78067324459991
MFI: 76.68189377401848
EMA 1234.4026279753841
SO 47.24191725254059
RSI: 72.45949638293045
MFI: 81.89650038556738
EMA 1258.6589051828328
SO 12.98082321882248
RSI: 72.75950624023397
MFI: 72.88516268041757
EMA 1279.4696676949134
SO 56.87192678451538
RSI: 71.63376233350517
MFI: 72.63667436532215
EMA 1297.3471657783
SO 20.653527975082397
RSI: 71.205356

RSI: 50.01799278930481
MFI: 36.77077943231771
EMA 2100.3582185259447
SO 41.41682982444763
RSI: 50.965298311283604
MFI: 42.3526218098942
EMA 2104.070449545402
SO 81.50933980941772
RSI: 49.7395901508754
MFI: 47.80927384414495
EMA 2105.141069293515
SO 25.454819202423096
RSI: 47.919107717539966
MFI: 48.079940828778014
EMA 2101.42226005438
SO 20.404237508773804
RSI: 44.33750324430523
MFI: 42.1756148968822
EMA 2098.919279026296
SO 62.43886351585388
RSI: 46.2779736406485
MFI: 41.22425545337067
EMA 2100.010054843623
SO 65.67925214767456
RSI: 52.14700270632236
MFI: 50.180339253016854
EMA 2108.18872721864
SO 92.5304651260376
RSI: 61.13621255937069
MFI: 64.92008959825972
EMA 2135.6302302561544
SO 81.75119161605835
RSI: 60.58907756823673
MFI: 71.8904058574888
EMA 2160.1795328886674
SO 64.33477997779846
RSI: 59.92757656030708
MFI: 65.80091294648201
EMA 2179.162268347262
SO 26.34621560573578
RSI: 57.07464224713064
MFI: 59.43311878391747
EMA 2197.7873122551273
SO 81.64924383163452
RSI: 55.07763662498

RSI: 49.07405828993174
MFI: 39.53049873874247
EMA 1954.3977587577554
SO 82.28099942207336
RSI: 52.93210399821621
MFI: 46.857056766365744
EMA 1962.6647275119296
SO 59.834980964660645
RSI: 51.93165063875014
MFI: 53.277810970438715
EMA 1974.9360906665893
SO 94.16952133178711
RSI: 56.19199471852872
MFI: 58.9888728334764
EMA 1991.5312655568773
SO 54.22200560569763
RSI: 56.20447497859353
MFI: 58.01246329141774
EMA 1999.2737765034606
SO 18.0175319314003
RSI: 47.74807196479432
MFI: 54.3174220492076
EMA 1991.5039396363325
SO 8.742111921310425
RSI: 48.2546005778448
MFI: 54.84007184117418
EMA 1978.6367476848216
SO 18.918919563293457
RSI: 46.94777743821479
MFI: 54.70314892022661
EMA 1965.0251878372621
SO 12.289010733366013
RSI: 49.19953332028438
MFI: 47.501684756373585
EMA 1948.6151562818773
SO 14.380210638046265
RSI: 49.83634484499581
MFI: 54.217676887863
EMA 1941.5064752880437
SO 82.98254013061523
RSI: 47.24159213577918
MFI: 54.182497305805896
EMA 1939.4122850933882
SO 83.30400586128235
RSI: 42.